FaceValue Bioinstrumentation Project | Coded by Cata Santos Anzola | Finished 26/11/2025

Step 0. Import libraries

In [8]:
import numpy as np
import pandas as pd
from Arduino import Arduino
import time
import pickle
import joblib
from keras.models import load_model
import serial
import sys
import time

Step 1. Connect with Arduino

In [ ]:
# begin arduino connection -> will be bypassing use of actual arduino coding system through python's arduino library
number = 3; #CHANGE THIS TO THE ACTUAL PIN FOR THE ARDUINO
board = Arduino(number, port="COM3") # connect to arduino , make sure COM and number is fine
# we will be usiing the Arduino to send the string output to the LCD screen

Step 2. Receive SD Card Data

In [ ]:
# grab SD file from arduino  --> if change the directory, change here too
path = "C:/Users/catas\OneDrive - Clemson University/Uni/Bioinstrumentation Lab/FaceValue/ArduinoEMGCode"
data = read_txt_file(path)
data = np.array(data).reshape(1, -1)  # reshape for model input

Step 3. Run EMG signal into ML model for classification

In [ ]:
#import ML model, saved after training yipee
model = load_model('C:/Users/catas/OneDrive - Clemson University/Uni/Bioinstrumentation Lab/FaceValue/EMGClassifier.hdf5')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
outcome=model.predict(data)
print(outcome)

#change the comparison into watever is read as ALS positive from the model
if outcome != 0:
    stringa = 'ALS Suspected'
else:
    stringa = 'ALS Not Suspected'

Step 4. output string from model result to arduino

In [ ]:
board.write(bytes(stringa, 'utf-8'))
board.close()

           ,....,
        ,;;:o;;;o;;,
      ,;;o;'''''';;;;,
     ,;:;;        ;;o;,
     ;o;;          ;;;;        Happy Holidays <3
     ';;;,  _  _  ,;;;'
      ';o;;/_\/_\;;o;'
         ';;\_\/_/;;'
           '//\\'
           //  \\
          |/    \|